In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, Input, layers, optimizers, regularizers, callbacks, losses, metrics 
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, train_test_split

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
# FOR KAGGLE
train_images_dir = '../input/petfinder-pawpularity-score/train/'
test_images_dir = '../input/petfinder-pawpularity-score/test/'

# FOR LOCAL PC
# train_images_dir = './dataset/train/'
# test_images_dir = './dataset/test/'

In [11]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df.loc[:, 'filename'] = train_images_dir + train_df['Id'] + '.jpg'

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df.loc[:, 'filename'] = test_images_dir + test_df['Id'] + '.jpg'

In [26]:
train_df.head(5)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(train_df.loc[:, train_df.columns != 'Pawpularity'], train_df['Pawpularity'], test_size=0.2, random_state=42)

In [14]:
cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [15]:
img_size = (128, 128, 3)
BATCH_SIZE = 32

def parse_image(inputs, label=None):
    image_string = tf.io.read_file(inputs[0])
    image = tf.image.decode_jpeg(image_string, channels=img_size[2])
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, img_size[:2])
    return (image, inputs[1]), label

def get_dataset(inputs, label=None, shuffle=False):    
    dataset = tf.data.TFRecordDataset.from_tensor_slices((inputs, label))     
    if shuffle:
        dataset = dataset.shuffle(np.size(inputs[0], 0))
    dataset = dataset.map(parse_image, num_parallel_calls=4)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(True)
    return dataset

In [16]:
train_dataset = get_dataset((X_train['filename'], X_train[cols]), y_train, shuffle=True)
val_dataset = get_dataset((X_val['filename'], X_val[cols]), y_val, shuffle=True)
test_dataset = get_dataset((test_df['filename'], test_df[cols]))

In [18]:
# inputs
image_input = Input(shape=img_size)
tabular_input = Input(shape=(len(cols),))

# image processing
x = layers.Conv2D(16, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(image_input)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(image_input)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Dropout(0.2)(x)
flat_img = layers.Flatten()(x)

# tabular metadata processing
y = layers.Dense(128, activation='relu')(tabular_input)
y = layers.Dropout(0.3)(y)
y = layers.Dense(64, activation='relu')(y)
y = layers.Dropout(0.3)(y)

# combining
z = layers.Concatenate(name='concat')([flat_img, y])
z = layers.Dense(32, activation='relu')(z)
z = layers.Dropout(0.3)(z)
z = layers.Dense(16, activation='relu')(z)
output = layers.Dense(1)(z)
# output = layers.Rescaling(100)(z)

model = tf.keras.Model(
    inputs = (image_input, tabular_input),
    outputs=output
)
model.compile(optimizer=optimizers.Adam(), 
              loss=losses.MeanSquaredError(), 
              metrics=[metrics.RootMeanSquaredError()]
             )

In [19]:
es = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.002, patience=10)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=0.000001)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=128,
#     steps_per_epoch=20,
#     use_multiprocessing=True,
#     callbacks=[es, reduce_lr]
)

In [20]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [21]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [22]:
preds = model.predict(test_dataset)

In [23]:
submisson_df = test_df[['Id']]
submisson_df['Pawpularity'] = preds

In [25]:
submisson_df

In [ ]:
submisson_df.to_csv('submission.csv', index=True)